In [2]:
from bs4 import BeautifulSoup
import requests

from datetime import datetime
from dateutil.parser import parse
import time
from selenium import webdriver
import undetected_chromedriver as uc 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

from pprint import pprint
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import pandas as pd

In [3]:
# class to determine ganjil genap
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False

In [135]:
# reading excel file
df = pd.read_excel('C:/Users/Lenovo/Documents/nabati/export shipment tracker/shipment_tracker/BL Numbers.xlsx') 
parse_bl = df['SINOKOR'].tolist()
bl_list = []
for i in parse_bl:
    bl_list.append(str(i))
bl_list = [x.replace(' ', '') for x in bl_list]
bl_list = [x for x in bl_list if x != 'nan']

In [136]:
s = slice(10)
bl_list


['SNKO073230801395', 'SNKO073230801128', 'SNKO073230801008']

In [143]:
bl_list=["SNKO073230500252",
"SNKO073230500325",
"SNKO073230500421",
"SNKO073230500515",
"SNKO073230500849",
"SNKO073230600199",
"SNKO073230600342"
]

Data yang di ambil hanya dari deprture ke bawah

In [137]:
def check_for_kw(word_list, input_string):
    for word in word_list:
        if word in input_string:
            return True
    return False

In [144]:
list_of_dict = []

# web driver
options = Options()
options.add_argument("--window-size=1920,1280")
driver = uc.Chrome()

for i, bls in enumerate(bl_list):
    # acessing BL
    driver.get("http://ebiz.sinokor.co.kr/BLDetail?blno=" + bls)

    time.sleep(1)
    
    # getting containers in bl
    soup = BeautifulSoup(driver.page_source, 'lxml')

    # getting from to of the shipment
    places = soup.find_all('div', {'class', 'col-sm-6 col-md-6'}) 
    origin_city = places[0].text.replace('\n','').replace('  ','').split(" ")[0]
    origin_port = places[0].text.replace('\n','').replace('  ','').split(" ")[1]
    destination_city = places[1].text.replace('\n','').replace('  ','').split(" ")[0]
    destination_port = places[1].text.replace('\n','').replace('  ','').split(" ")[1]

    # list of containers in 1 bl
    container_table = soup.find_all('table', {'class', 'table detailTable dataTable no-footer'}) 
    list_of_containers = []
    for tag in container_table:
        for c, ctr  in enumerate(tag.find_all('tr', {"class":'odd'}) + tag.find_all('tr', {"class":'even'})):
            for c2, ctr2 in enumerate(ctr.find_all('td')):
                if len(ctr2.text) == 11:
                    list_of_containers.append(ctr2.text)
    
    # milestons of shipment
    milestone_table = soup.find_all('div', {'class', 'splitTable'})

    # appending milestones to list
    list_all = []
    for tag2 in milestone_table:
        for m1, milestone1  in enumerate(tag2.find_all('tr')):
            for m2, theader in enumerate(milestone1.find_all('th',{'class', 'firstTh'})):
                list_all.append(theader.text)
            for m3, milestone2 in enumerate(milestone1.find_all('td')):
                list_all.append(milestone2.text)
    
    # splitting list
    for li, element in enumerate(list_all):
        if element == "Departure":
            dept_temp = li
        elif element == "Arrival":
            arrival_temp = li
        elif element.split(" ")[0] == "Pickup":
            pickup_temp = li
        elif element.split(" ")[0] == "Return":
            return_temp = li

    # list/movement
    departure_milestone = list_all[dept_temp:arrival_temp]
    arrival_milestone = list_all[arrival_temp:pickup_temp]
    pickup_milestone = list_all[pickup_temp:return_temp]
    return_milestone = list_all[return_temp:]

    # pickup and return dictionaries
    pickup_dict = {}
    for p, pickup_date in enumerate(pickup_milestone):
        if is_date(pickup_date):
            case_pickup = {pickup_milestone[p-2]:pickup_date}
            pickup_dict.update(case_pickup)

    return_dict = {}
    for r, return_date in enumerate(return_milestone):
        if is_date(return_date):
            case_return = {return_milestone[r-2]:return_date}
            return_dict.update(case_return)

    for c, container in enumerate(list_of_containers):
        current_dict = {}
        try:
            current_dict.update({"Liners":"SINOKOR"})
            current_dict.update({"BL Number":bls})
            current_dict.update({"Container Number":container})
            current_dict.update({"From": origin_city})
            current_dict.update({"To": destination_city})
            current_dict.update({"ATD":departure_milestone[3][s]})
            current_dict.update({"ATA":arrival_milestone[3][s]})
            current_dict.update({"Container Release":pickup_dict[container][s]})
            current_dict.update({"Container Return":return_dict[container][s]})
        except:
            pass

        list_of_dict.append(current_dict)


In [145]:
list_of_dict

[{'Liners': 'SINOKOR',
  'BL Number': 'SNKO073230500252',
  'Container Number': 'HALU5641780',
  'From': 'Jakarta',
  'To': 'Hochiminh',
  'ATD': '2023-05-10',
  'ATA': '2023-05-15',
  'Container Release': '2023-06-04',
  'Container Return': '2023-06-19'},
 {'Liners': 'SINOKOR',
  'BL Number': 'SNKO073230500252',
  'Container Number': 'HLHU8115297',
  'From': 'Jakarta',
  'To': 'Hochiminh',
  'ATD': '2023-05-10',
  'ATA': '2023-05-15',
  'Container Release': '2023-06-04',
  'Container Return': '2023-06-16'},
 {'Liners': 'SINOKOR',
  'BL Number': 'SNKO073230500325',
  'Container Number': 'BMOU4506473',
  'From': 'Jakarta',
  'To': 'Shanghai',
  'ATD': '2023-05-16',
  'ATA': '2023-05-28',
  'Container Release': '2023-06-03',
  'Container Return': '2023-06-03'},
 {'Liners': 'SINOKOR',
  'BL Number': 'SNKO073230500325',
  'Container Number': 'HALU5648147',
  'From': 'Jakarta',
  'To': 'Shanghai',
  'ATD': '2023-05-16',
  'ATA': '2023-05-28',
  'Container Release': '2023-06-05',
  'Containe

In [147]:
import datetime
df = pd.DataFrame(list_of_dict)

excel_file_path = 'export excel/SINOKOR.xlsx'

df.to_excel(excel_file_path, index=False)

In [67]:
# getting from to of the shipment
places = soup.find_all('div', {'class', 'col-sm-6 col-md-6'}) 
origin_city = places[0].text.replace('\n','').replace('  ','').split(" ")[0]
origin_port = places[0].text.replace('\n','').replace('  ','').split(" ")[1]
destination_city = places[1].text.replace('\n','').replace('  ','').split(" ")[0]
destination_port = places[1].text.replace('\n','').replace('  ','').split(" ")[1]

In [69]:
# list of containers in 1 bl
container_table = soup.find_all('table', {'class', 'table detailTable dataTable no-footer'}) 
list_of_containers = []
for tag in container_table:
    for c, ctr  in enumerate(tag.find_all('tr', {"class":'odd'}) + tag.find_all('tr', {"class":'even'})):
        for c2, ctr2 in enumerate(ctr.find_all('td')):
            if len(ctr2.text) == 11:
                list_of_containers.append(ctr2.text)

In [70]:
# milestons of shipment
milestone_table = soup.find_all('div', {'class', 'splitTable'})

In [72]:
list_all = []
for tag2 in milestone_table:
    for m1, milestone1  in enumerate(tag2.find_all('tr')):
        for m2, theader in enumerate(milestone1.find_all('th',{'class', 'firstTh'})):
            list_all.append(theader.text)
        for m3, milestone2 in enumerate(milestone1.find_all('td')):
            list_all.append(milestone2.text)

In [89]:
for li, element in enumerate(list_all):
    if element == "Departure":
        dept_temp = li
    elif element == "Arrival":
        arrival_temp = li
    elif element.split(" ")[0] == "Pickup":
        pickup_temp = li
    elif element.split(" ")[0] == "Return":
        return_temp = li


In [117]:
departure_milestone = list_all[dept_temp:arrival_temp]
arrival_milestone = list_all[arrival_temp:pickup_temp]
pickup_milestone = list_all[pickup_temp:return_temp]
return_milestone = list_all[return_temp:]

In [131]:
departure_milestone

['Departure',
 'STARSHIP JUPITER / 2303N',
 'TPK (TERMINAL PETIKEMAS KOJA)',
 '2023-08-29 TUE 09:00']

In [122]:
pickup_dict = {}
for p, pickup_date in enumerate(pickup_milestone):
    if is_date(pickup_date):
        case_pickup = {pickup_milestone[p-2]:pickup_date}
        pickup_dict.update(case_pickup)

return_dict = {}
for r, return_date in enumerate(return_milestone):
    if is_date(return_date):
        case_return = {return_milestone[r-2]:return_date}
        return_dict.update(case_return)

In [123]:
return_dict

{'SKHU9968460': '2023-09-07 THU 18:57',
 'HLHU8118634': '2023-09-10 SUN 00:10',
 'BMOU5948361': '2023-09-10 SUN 12:43',
 'BEAU4213362': '2023-09-10 SUN 14:15',
 'HLHU8101502': '2023-09-10 SUN 14:26',
 'HLHU8100511': '2023-09-10 SUN 16:04',
 'HLHU8206205': '2023-09-10 SUN 22:36',
 'HLHU8206653': '2023-09-11 MON 08:38',
 'SKHU8933562': '2023-09-12 TUE 12:24',
 'SKHU8121208': '2023-09-12 TUE 14:23',
 'UETU5880873': '2023-09-12 TUE 14:38',
 'SKHU9806639': '2023-09-12 TUE 14:39'}

In [132]:
list_of_dict = []
for c, container in enumerate(list_of_containers):
    current_dict = {}
    try:
        current_dict.update({"Liners":"SINOKOR"})
        # current_dict.update({"BL Number":bls)
        current_dict.update({"Container Number":container})
        current_dict.update({"From": origin_city})
        current_dict.update({"To": destination_city})
        current_dict.update({"ATD":departure_milestone[3][s]})
        current_dict.update({"ATA":arrival_milestone[3][s]})
        current_dict.update({"Container Release":pickup_dict[container][s]})
        current_dict.update({"Container Return":return_dict[container][s]})
    except:
        pass
    list_of_dict.append(current_dict)

In [134]:
list_of_dict

[{'Liners': 'SINOKOR',
  'Container Number': 'BEAU4213362',
  'From': 'Jakarta',
  'To': 'Shekou',
  'ATD': '2023-08-29',
  'ATA': '2023-09-05',
  'Container Release': '2023-09-09',
  'Container Return': '2023-09-10'},
 {'Liners': 'SINOKOR',
  'Container Number': 'HLHU8100511',
  'From': 'Jakarta',
  'To': 'Shekou',
  'ATD': '2023-08-29',
  'ATA': '2023-09-05',
  'Container Release': '2023-09-10',
  'Container Return': '2023-09-10'},
 {'Liners': 'SINOKOR',
  'Container Number': 'HLHU8118634',
  'From': 'Jakarta',
  'To': 'Shekou',
  'ATD': '2023-08-29',
  'ATA': '2023-09-05',
  'Container Release': '2023-09-09',
  'Container Return': '2023-09-10'},
 {'Liners': 'SINOKOR',
  'Container Number': 'HLHU8206653',
  'From': 'Jakarta',
  'To': 'Shekou',
  'ATD': '2023-08-29',
  'ATA': '2023-09-05',
  'Container Release': '2023-09-09',
  'Container Return': '2023-09-11'},
 {'Liners': 'SINOKOR',
  'Container Number': 'SKHU8933562',
  'From': 'Jakarta',
  'To': 'Shekou',
  'ATD': '2023-08-29',
  

In [51]:
milestone_table

[<div class="splitTable">
 <table>
 <tbody><tr>
 <th class="firstTh" rowspan="5"><span>Return (4/4)</span></th>
 <th>CNTR No.</th>
 <th>Location</th>
 <th>Date &amp; Time</th>
 </tr>
 <tr>
 <td>TRHU5289277</td>
 <td>JICT (JKT INTERNATIONAL CONTAINER Termial)</td>
 <td>2023-08-29 TUE 03:56</td>
 </tr>
 <tr>
 <td>GESU6620277</td>
 <td>JICT (JKT INTERNATIONAL CONTAINER Termial)</td>
 <td>2023-08-29 TUE 12:53</td>
 </tr>
 <tr>
 <td>SKHU8107299</td>
 <td>JICT (JKT INTERNATIONAL CONTAINER Termial)</td>
 <td>2023-08-30 WED 00:18</td>
 </tr>
 <tr>
 <td>SKHU9919609</td>
 <td>JICT (JKT INTERNATIONAL CONTAINER Termial)</td>
 <td>2023-08-30 WED 06:44</td>
 </tr>
 <tr>
 <th class="firstTh" rowspan="2"><span>Departure</span></th>
 <th>Vessel / Voyage</th>
 <th>Location</th>
 <th>Date &amp; Time</th>
 </tr>
 <tr>
 <td>SAWASDEE SUNRISE / 2309N</td>
 <td>JICT (JKT INTERNATIONAL CONTAINER Termial)</td>
 <td>2023-08-31 THU 23:18</td>
 </tr>
 <tr>
 <th class="firstTh" rowspan="2"><span>Arrival</span></th>

In [7]:
data_cut = []
satu_tanggal = []
result_3 = {}
final = []
key_word = "Departure"
kw = ["Departure Vessel", "Arrival Vessel", "Pickup ", "Return "]

# web scripting
options = Options()
options.add_argument("--window-size=1920,1280")
driver = uc.Chrome()

for i, bls in enumerate(bl_list):
    # acessing BL
    driver.get("http://ebiz.sinokor.co.kr/BLDetail?blno=" + bls)

    # scraping table
    table = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div[5]/div[2]/div/div/div/table')
    data_tr = table.find_elements(By.TAG_NAME, "tr")

    # converting list of web elements to list of text
    data_text = [data.text for data in data_tr]
    for j, item in enumerate(data_text):
        if key_word in item:
            tanda = j
            data_cut = data_text[tanda:]

    # appending {keterangan : tanggal}
    for k, date in enumerate(data_cut):
        if check_for_kw(kw, date):
            case = {date:data_cut[k+1][-20:-10]}
            satu_tanggal.append(case)
    
    # adding BL number
    result_3[bl_list[i]] = satu_tanggal
    satu_tanggal=[]
    
final.append(result_3)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div/div[3]/div[5]/div[2]/div/div/div/table"}
  (Session info: chrome=116.0.5845.180); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6EDA952A2+57122]
	(No symbol) [0x00007FF6EDA0EA92]
	(No symbol) [0x00007FF6ED8DE3AB]
	(No symbol) [0x00007FF6ED917D3E]
	(No symbol) [0x00007FF6ED917E2C]
	(No symbol) [0x00007FF6ED950B67]
	(No symbol) [0x00007FF6ED93701F]
	(No symbol) [0x00007FF6ED94EB82]
	(No symbol) [0x00007FF6ED936DB3]
	(No symbol) [0x00007FF6ED90D2B1]
	(No symbol) [0x00007FF6ED90E494]
	GetHandleVerifier [0x00007FF6EDD3EF82+2849794]
	GetHandleVerifier [0x00007FF6EDD91D24+3189156]
	GetHandleVerifier [0x00007FF6EDD8ACAF+3160367]
	GetHandleVerifier [0x00007FF6EDB26D06+653702]
	(No symbol) [0x00007FF6EDA1A208]
	(No symbol) [0x00007F

In [92]:
final

[{'SNKO073230600199': [{'Departure Vessel / Voyage Location Date & Time': '2023-06-13'},
   {'Arrival Vessel / Voyage Location Date & Time': '2023-06-25'},
   {'Pickup (2/2) CNTR No. Location Date & Time': '2023-06-28'},
   {'Return (2/2) CNTR No. Location Date & Time': '2023-06-28'}],
  'SNKO073230600342': [{'Departure Vessel / Voyage Location Date & Time': '2023-06-16'},
   {'Arrival Vessel / Voyage Location Date & Time': '2023-06-23'},
   {'Pickup (9/9) CNTR No. Location Date & Time': '2023-06-25'},
   {'Return (9/9) CNTR No. Location Date & Time': '2023-06-26'}],
  'SNKO073230600271': [{'Departure Vessel / Voyage Location Date & Time': '2023-06-16'},
   {'Arrival Vessel / Voyage Location Date & Time': '2023-06-20'},
   {'Pickup (3/3) CNTR No. Location Date & Time': '2023-07-02'},
   {'Return (3/3) CNTR No. Location Date & Time': '2023-07-04'}],
  'SNKO073230600671': [{'Departure Vessel / Voyage Location Date & Time': '2023-06-19'},
   {'Arrival Vessel / Voyage Location Date & Time'